# Divide data into test/training sets

In [1]:
from pathlib import Path
import pickle

In [2]:
import json
config = json.load((Path.cwd().parent / 'config.json').open('rb'))
calibration_fp = Path(config['project_directory']) / 'Calibration'
cycleatl_fp = Path(config['project_directory']) / 'CycleAtlanta'
matching_fp = Path(config['project_directory']) / 'Map_Matching'
network_fp = Path(config['project_directory']) / 'Network'
if calibration_fp.exists() == False:
    calibration_fp.mkdir()

# Import Matched Trace Data
Dictionary Containing Origins and Destinations, Matches, Shortest Paths

In [3]:
#export for impedance calibration
with (calibration_fp/'ready_for_calibration.pkl').open('rb') as fh:
    match_results = pickle.load(fh)
with (cycleatl_fp/'trips_3.pkl').open('rb') as fh:
    reduced_trip_set = pickle.load(fh)
filter_trips = set(reduced_trip_set['tripid'].tolist())

In [4]:
match_results = {key:item for key, item in match_results.items() if key in filter_trips}
print(len(match_results))

1825


## Split the data into training and testing set
For starters, lets select 500 random trips to go in each

In [5]:
import random
def get_random_keys(dictionary,num_keys):
    random_keys = set()
    while len(random_keys) < num_keys:
        random_key =  random.choice(list(dictionary.keys()))
        if isinstance(dictionary.get(random_key),str):
            continue
        if random_key not in random_keys:
            random_keys.add(random_key)
    return list(random_keys)

In [6]:
total_trips = 1000
random_keys = get_random_keys(match_results,total_trips)

middle_point = int(total_trips/2)

train_set = random_keys[0:middle_point]
test_set = random_keys[middle_point:]

In [7]:
#subset accordingly
train_set = {key:item for key, item in match_results.items() if key in train_set}
test_set = {key:item for key, item in match_results.items() if key in test_set}

In [8]:
with (calibration_fp/'test_set.pkl').open('wb') as fh:
    pickle.dump(train_set,fh)
with (calibration_fp/'train_set.pkl').open('wb') as fh:
    pickle.dump(test_set,fh)